# Summary
This script is written to transform training images for transfer learning using the output bounding box calculated using YOLO

In [ ]:
import os
import pathlib
from PIL import Image

In [ ]:
# Changeable Param
train_dir = r'D:\Resources\Inat_Partial\Aves_Small_SS1_Train\CV_0'
output_dir = r'D:\Resources\Inat_Partial\Aves_Small_SS1_Train_YOLO\CV_0'
yolo_output_csv = r'D:\Workspace\Jupyter\COMP5703\Resources\ss1_bbox.csv'

In [ ]:
if(os.path.isdir(output_dir) == False):
    pathlib.Path(output_dir).mkdir(parents=True, exist_ok=True)

In [ ]:
def crop(image_path, coords, saved_location):
    """
    @param image_path: The path to the image to edit
    @param coords: A tuple of x/y coordinates (x1, y1, x2, y2)
    @param saved_location: Path to save the cropped image
    """
    try:
        image_obj = Image.open(image_path)
        cropped_image = image_obj.crop(coords)
        cropped_image.save(saved_location)
    except Exception as e:
        pass

In [ ]:
with open(yolo_output_csv, 'r', encoding='utf-8') as yoc:
    prev_index = 0
    prev_image_name = ''
    for i,line in enumerate(yoc):
        if i % 1000 == 0:
            print('Parsing line',i)
        subdir, image_name, top, left, bottom, right, score = line.split(',')
        top, left, bottom, right = [int(coor) for coor in (top, left, bottom, right)]
        score = float(score)
        train_image_path = os.path.join(train_dir, subdir, image_name)
        output_subdir_path = os.path.join(output_dir, subdir)
        # In case that one image has more than one bounding boxes, the next one has to be saved in a different name
        if prev_image_name == image_name:
            # If prev image is abcd.jpg, the next one will be saved as abcd(1).jpg and so on
            custom_image_name = (image_name.split('.')[0] + r'(' + 
                                 str(prev_index) + r')' + r'.' + 
                                 image_name.split('.')[1])
            output_image_path = os.path.join(output_dir, subdir, custom_image_name)
        else:
            output_image_path = os.path.join(output_dir, subdir, image_name)
        
        if(os.path.isdir(output_subdir_path) == False):
            os.mkdir(output_subdir_path)
        crop(train_image_path, (left,top,right,bottom), output_image_path)
        
        prev_index += 1
        prev_image_name = image_name